In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error
from scipy.stats import pearsonr,norm

In [ ]:
train=pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/train.csv")
test=pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv')
train.head()
train.drop('id',axis=1,inplace=True)


In [ ]:
fig,axes=plt.subplots(4,4,figsize=(15,15))
row=0
col=0
for column in train.columns:
    sns.distplot(train[column],ax=axes[row,col],kde=False)
    col+=1
    if col==4:
        row+=1
        col=0
fig.show()

In [ ]:
pvals=[]
cor=[]
for x in train.columns:
    r,pval=pearsonr(train[x],train['target'])
    pvals.append(pval)
    cor.append(r)
print(pvals)
cor.pop()
fig=plt.figure(figsize=(10,8))
sns.barplot(x=train.columns[:-1],y=cor)
plt.show()

In [ ]:
fig=plt.figure(figsize=(10,8))
sns.heatmap(train.corr())
fig.show()

In [ ]:
fig,axes=plt.subplots(4,4,figsize=(10,10))
row=0
col=0
for column in train.columns:
    sns.scatterplot(ax=axes[row,col],y=train[column],x=train['target'])
    col+=1
    if col==4:
        row+=1
        col=0
fig.show()

A single variable does not show any corelation with the target.

In [ ]:
sns.scatterplot(x='cont1',y='cont3',data=train)

In [ ]:
def quantile_plots(col1,col2,data,bins):
    data['bins'],xlabel=pd.qcut(data[col1],bins,retbins=True)
    df=data.groupby('bins')[col2]
    df=pd.Series(df)
    q0=[]
    q25=[]
    q50=[]
    q75=[]
    q100=[]
    for x in df: 
        values=np.quantile(x[1],[0.25,0.5,0.75])
        q0.append(min(x[1]))
        q25.append(values[0])
        q50.append(values[1])
        q75.append(values[2])
        q100.append(max(x[1]))
    fig=plt.figure()
    xlabel=list(xlabel)
    xlabel.pop()
    sns.lineplot(xlabel,q0,label='minimum value')
    sns.lineplot(xlabel,q25,label='25th percentile')
    sns.lineplot(xlabel,q50,label='50th percentil')
    sns.lineplot(xlabel,q75,label='75th percentile')
    sns.lineplot(xlabel,q100,label='100th percentile')
    plt.xlabel(col1)
    plt.legend()
    data.drop('bins',axis=1,inplace=True)
    plt.ylabel('Percentiles of '+col2)
    return fig

In [ ]:
quantile_plots('cont1','target',train,20).show()

In [ ]:
forest=RandomForestRegressor(max_depth=3,n_estimators=1000)
forest.fit(train.drop('target',axis=1),train['target'])

In [ ]:
train_pred=forest.predict(train.drop('target',axis=1))
mean_absolute_error(train_pred,train['target'])

In [ ]:
pred=forest.predict(test.drop('id',axis=1))
output=pd.DataFrame({'id':test['id'],'target':pred})
output.set_index('id',inplace=True)
output.to_csv('output.csv')

In [ ]:
output.head()